Thêm thư viện

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import math
warnings.filterwarnings("ignore")

Load data từ BikeSharingDemand

In [2]:
# Đọc dữ liệu 
dt = pd.read_csv("BikeSharingDemand.csv")
print ( "Hiển thị 5 dòng đầu của tập data\n ", dt.head())
print ( "Xem kiểu dữ liệu của từng thuộc tính ", dt.dtypes)
print ( "Xem thông tin của dữ liệu ", dt.info())
print ( "Kiểm tra giá trị thiếu ",dt.isnull().sum())

Hiển thị 5 dòng đầu của tập data
          datetime  season  holiday  workingday  weather  temp   atemp  \
0  1/1/2011 0:00       1        0           0        1  9.84  14.395   
1  1/1/2011 1:00       1        0           0        1  9.02  13.635   
2  1/1/2011 2:00       1        0           0        1  9.02  13.635   
3  1/1/2011 3:00       1        0           0        1  9.84  14.395   
4  1/1/2011 4:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  
Xem kiểu dữ liệu của từng thuộc tính  datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windsp

Chú thích cho tập dữ liệu  <br>
datetime: cho biết thời gian theo dịnh dạng "%m/%d/%Y %H:%M:%hS  <br> 
season: cho biết mùa  <br> 
1: mùa xuân  <br>
2: mùa hè  <br>
3: mùa thu  <br>
4: mùa đông  <br>
workingday: cho biết có phải là ngày làm việc không bao gồm cả cuối tuần và ngày lễ  <br> 
1: ngày làm việc  
0: ngày không làm việc 
weather: Cho biết thời tiết  <br>
1: Trời trong, có mây, có mây rải rác  <br>
2: Sương mù + Đầy mây, sương mù + ít mây, sương mù  <br>
3: Tuyết nhẹ, mưa nhẹ + sấm sét + mây rải rác, mưa nhẹ + mây rải rác  <br>
4: Mưa lớn + mảnh băng + sấm sét + sương mù, tuyết + sương mù  <br>
temp: Nhiệt độ thực tế tính theo độ Celsius (độ C)  <br>
atemp: Nhiệt độ mà con người cảm thấy tính theo độ Celsius (độ C) <br>
humidity: Độ ẩm tương đối <br>
windspeed: Tốc độ gió <br>
casual: Số lượng xe thuê mà không có đăng ký dịch vụ <br>
registered: Số lượng xe thuê mà có đăng ký dịch vụ <br>
count: Tổng số lượng xe được thuê <br>

Xử lý Data

In [3]:
# tạo cột year, month, day, hour,weekday từ cột datetime
dt["datetime"] = pd.to_datetime(dt["datetime"])
dt["year"] = dt["datetime"].dt.year
dt["month"] = dt["datetime"].dt.month
dt["weekday"] = dt["datetime"].dt.weekday
dt["hour"] = dt["datetime"].dt.hour
print(" Hiển thị 5 dòng đầu của tập data\n",  dt.head())
print(" Xem kiểu dữ liệu của từng thuộc tính",  dt.dtypes)
print(" Xem thông tin của dữ liệu",  dt.info())
print(" Kiểm tra giá trị thiếu", dt.isnull().sum())

 Hiển thị 5 dòng đầu của tập data
              datetime  season  holiday  workingday  weather  temp   atemp  \
0 2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1 2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2 2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3 2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4 2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  year  month  weekday  hour  
0        81        0.0       3          13     16  2011      1        5     0  
1        80        0.0       8          32     40  2011      1        5     1  
2        80        0.0       5          27     32  2011      1        5     2  
3        75        0.0       3          10     13  2011      1        5     3  
4        75        0.0       0           1      1  2011      1        5     4  
 Xem kiểu dữ liệu của 

Chú thích các thuộc tính mới  <br>
month: tháng (1-12)  <br>
year: năm (2011, 2012)  <br>
hour: giờ trong ngày (0-23 giờ)  <br>
weekday: ngày trong tuần  <br> 
0: thứ 2  <br>
1: thứ 3  <br>
2: thứ 4  <br>
3: thứ 5  <br>
4: thứ 6  <br>
5: thứ 7  <br>
6: chủ nhật  <br>

Chia Dữ tập dữ liệu theo Hold-Out <br>
Test = 1/3 , train = 2/3  <br>

In [4]:
# Chia dữ liệu thành X và y
X = dt.drop(columns=["casual", "registered", "count", "datetime"])
print(X)
y = dt["count"] 
print(y)
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1 / 3, random_state=42
)
print(X_train)
print(X_test)

       season  holiday  workingday  weather   temp   atemp  humidity  \
0           1        0           0        1   9.84  14.395        81   
1           1        0           0        1   9.02  13.635        80   
2           1        0           0        1   9.02  13.635        80   
3           1        0           0        1   9.84  14.395        75   
4           1        0           0        1   9.84  14.395        75   
...       ...      ...         ...      ...    ...     ...       ...   
10881       4        0           1        1  15.58  19.695        50   
10882       4        0           1        1  14.76  17.425        57   
10883       4        0           1        1  13.94  15.910        61   
10884       4        0           1        1  13.94  17.425        61   
10885       4        0           1        1  13.12  16.665        66   

       windspeed  year  month  weekday  hour  
0         0.0000  2011      1        5     0  
1         0.0000  2011      1        5   

Dùng GridSearchCV để tỉm ra tham số tốt nhất cho mô hình <br>

In [5]:

# Định nghĩa các giá trị tham số để kiểm tra tham số tốt nhất
param_grid = {
    "n_neighbors": [
        1,
        3,
        5,
        7,
        9,
        11,
        13,
        15,
        17,
        19,
        21,
        23,
        25,
        27,
        29,
        31,
        33,
        35,
        37,
        39,
        41,
        43,
        45,
        47,
        49,
        51,
        53,
        55,
        57,
        59,
        61,
        63,
        65,
        67,
        69,
        71,
        73,
        75,
        77,
        79,
        81,
        83,
        85,
        87,
        89,
        91,
        93,
        95,
        97,
        99,
    ],
    "weights": ["uniform", "distance"],
    "p": [1, 2],
}

Huấn luyện bằng GridSearchCV

In [6]:
# Tạo GridSearchCV
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid)

# Huấn luyện GridSearchCV object
grid_search.fit(X_train, y_train)


GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21,
                                         23, 25, 27, 29, 31, 33, 35, 37, 39, 41,
                                         43, 45, 47, 49, 51, 53, 55, 57, 59, ...],
                         'p': [1, 2], 'weights': ['uniform', 'distance']})

In ra tham số tốt nhất 

In [7]:
# In ra các tham số tốt nhất
print("Best parameters found: ", grid_search.best_params_)

# Dự đoán kết quả trên tập kiểm tra với các tham số tốt nhất
y_pred_grid = grid_search.predict(X_test)



Best parameters found:  {'n_neighbors': 9, 'p': 1, 'weights': 'distance'}


Xây dựng mô hình bằng KNN với tham số tốt nhất từ GridSearchCV <br>

In [8]:
model = KNeighborsRegressor(n_neighbors=9, p=1, weights="distance")
model.fit(X_train, y_train)
# Dự đoán kết quả trên tập kiểm tra
y_pred = model.predict(X_test)

Đánh giá mô hình

In [9]:
print("MSE = ", mean_squared_error(y_test, y_pred))
print("RMSE = ", np.sqrt(mean_squared_error(y_test, y_pred)))

MSE =  12031.713359839296
RMSE =  109.68916701224099


Chia Dữ tập dữ liệu theo Kfold

Load Data

In [10]:
# Load dữ liệu từ file CSV
dt = pd.read_csv("BikeSharingDemand.csv")
print ( "Hiển thị 5 dòng đầu của tập data\n ", dt.head())
print ( "Xem kiểu dữ liệu của từng thuộc tính ", dt.dtypes)
print ( "Xem thông tin của dữ liệu ", dt.info())
print ( "Kiểm tra giá trị thiếu ",dt.isnull().sum())

Hiển thị 5 dòng đầu của tập data
          datetime  season  holiday  workingday  weather  temp   atemp  \
0  1/1/2011 0:00       1        0           0        1  9.84  14.395   
1  1/1/2011 1:00       1        0           0        1  9.02  13.635   
2  1/1/2011 2:00       1        0           0        1  9.02  13.635   
3  1/1/2011 3:00       1        0           0        1  9.84  14.395   
4  1/1/2011 4:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  
Xem kiểu dữ liệu của từng thuộc tính  datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windsp

Xử lý Data

In [11]:
# tạo cột year, month, day, hour,weekday từ cột datetime
dt["datetime"] = pd.to_datetime(dt["datetime"])
dt["year"] = dt["datetime"].dt.year
dt["month"] = dt["datetime"].dt.month
dt["weekday"] = dt["datetime"].dt.weekday
dt["hour"] = dt["datetime"].dt.hour
print ( "Hiển thị 5 dòng đầu của tập data\n ", dt.head())
print ( "Xem kiểu dữ liệu của từng thuộc tính ", dt.dtypes)
print ( "Xem thông tin của dữ liệu ", dt.info())
print ( "Kiểm tra giá trị thiếu ",dt.isnull().sum())

Hiển thị 5 dòng đầu của tập data
               datetime  season  holiday  workingday  weather  temp   atemp  \
0 2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1 2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2 2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3 2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4 2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  year  month  weekday  hour  
0        81        0.0       3          13     16  2011      1        5     0  
1        80        0.0       8          32     40  2011      1        5     1  
2        80        0.0       5          27     32  2011      1        5     2  
3        75        0.0       3          10     13  2011      1        5     3  
4        75        0.0       0           1      1  2011      1        5     4  
Xem kiểu dữ liệu của t

Chia tập dữ liệu theo kfold 

In [12]:
# Chia dữ liệu thành X và y
X = dt.drop(columns=["casual", "registered", "count", "datetime"])
print(X)
y = dt["count"] 
print(y)
# Số lượng fold
n_splits = 10

# Khởi tạo KFold
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

       season  holiday  workingday  weather   temp   atemp  humidity  \
0           1        0           0        1   9.84  14.395        81   
1           1        0           0        1   9.02  13.635        80   
2           1        0           0        1   9.02  13.635        80   
3           1        0           0        1   9.84  14.395        75   
4           1        0           0        1   9.84  14.395        75   
...       ...      ...         ...      ...    ...     ...       ...   
10881       4        0           1        1  15.58  19.695        50   
10882       4        0           1        1  14.76  17.425        57   
10883       4        0           1        1  13.94  15.910        61   
10884       4        0           1        1  13.94  17.425        61   
10885       4        0           1        1  13.12  16.665        66   

       windspeed  year  month  weekday  hour  
0         0.0000  2011      1        5     0  
1         0.0000  2011      1        5   

Huấn luyện mô hình Theo KNeighborsRegressor

In [13]:
# Danh sách để lưu trữ MSE và RMSE cho từng mô hình
mse_results = []
rmse_results = []
# Lặp qua các fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Xây dựng và huấn luyện mô hình KNN Regression trên fold hiện tại
    model = KNeighborsRegressor(n_neighbors=9, p=1, weights="distance")
    model.fit(X_train, y_train)
    
    # Dự đoán kết quả trên fold kiểm tra
    y_pred = model.predict(X_test)
    # Đánh giá mô hình
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    mse_results.append(mse)
    rmse_results.append(rmse)
    

Giá giá mô hình

In [14]:
# Tính trung bình của MSE và RMSE từ các fold
avg_mse = sum(mse_results) / len(mse_results)
avg_rmse = sum(rmse_results) / len(rmse_results)

# In ra kết quả trung bình của MSE và RMSE
print("KNeighborsRegressor Average MSE = ", avg_mse)
print("KNeighborsRegressor Average RMSE = ", avg_rmse)

KNeighborsRegressor Average MSE =  11310.674490820833
KNeighborsRegressor Average RMSE =  106.30024405661182
